In [17]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

df = gpd.read_file('./ds2783.gdb/')


In [31]:
Tdata = df

print(Tdata[Tdata.index > 29456])

Tdata = Tdata.to_crs(epsg = 4326)

        RPT_YR SURVEY_ID1 SURVEY_ID2 SURVEY_ID3      DMG_TYPE1 DMG_TYPE2  \
29457     2006        FHP         -1         -1      Mortality   No Data   
29458     2006        FHP         -1         -1      Mortality   No Data   
29459     2006        FHP         -1         -1      Mortality   No Data   
29460     2006        FHP         -1         -1      Mortality   No Data   
29461     2006        FHP         -1         -1      Mortality   No Data   
...        ...        ...        ...        ...            ...       ...   
443416    2019       None       None       None  Mortality - 2      None   
443417    2019       None       None       None  Mortality - 2      None   
443418    2019       None       None       None  Mortality - 2      None   
443419    2019       None       None       None  Mortality - 2      None   
443420    2019       None       None       None  Mortality - 2      None   

       DMG_TYPE3 SEVERITY1 SEVERITY2 SEVERITY3  ...     ACRES  \
29457    No Data   No 

In [34]:

print(len(Tdata))
Tdata = Tdata.loc[Tdata["RPT_YR"] > 1949]
# state = Tdata[Tdata.STATE != "CA"]
# Tdata.drop(state.index)
Tdata = Tdata[Tdata['RPT_YR'].notna()]
Tdata.sort_values(by = ["Date"])
Tdata = Tdata.loc[Tdata["ACRES"] > 100]
Tdata = Tdata.loc[Tdata["SEVERITY1"] == "Moderate (11-29%)" or Tdata["SEVERITY1"] == "SEVERE (30-50%)"]
print(len(Tdata))
Tdata.tail

53314


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().